# Part 1

Implement a real-time Stock trading engine for matching Stock Buys with Stock Sells.<br>
1. Write an ‘addOrder’ function that will have the following parameters:<br>
      ‘Order Type’ (Buy or Sell), ‘Ticker Symbol’, ‘Quantity’, ‘Price’<br>
      Support 1,024 tickers (stocks) being traded.<br>
      Write a wrapper to have this ‘addOrder’ function randomly execute with different parameter values to simulate active stock transactions.<br>

2. Write a ‘matchOrder’ function, that will match Buy & Sell orders with the following criteria:<br>
      Buy price for a particular ticker is greater than or equal to lowest Sell price available then.<br>
      Write your code to handle race conditions when multiple threads modify the Stock order book, as run in real-life, by multiple stockbrokers. <br>
     

Program flow:<br><br>
1. The program begins by adding a Buy order or a Sell order to the respective order book (either buy_orders or sell_orders).<br>
2. After each order is added, the addOrder function sorts the order list—Buy orders in descending order by price, and Sell orders in ascending order by price. <br>
3. The matchOrder function is triggered to check for potential matches, where a Buy order with a price greater than or equal to the lowest Sell price for the same symbol can be matched. <br>
4. If a match is found, the orders are executed (quantities are adjusted), and the fully matched orders are removed from the lists. <br>
5. The process continues by adding new orders, attempting to match them, and printing the current status of the buy and sell orders in the order book.

In [1]:
# Load necessary libraries

# sleep for certain duration
import time
# to capture the datetime of order
import datetime
# Randomize the symbols, quantities, price
import random
import threading

In [2]:
# function to write any errors to log file
def log_error(error_message):
    """
    Function to write error to log file
    """
    with open("error_log.txt", "a") as f:
        # format with  current date and time along side with error message
        f.write(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {error_message}\n")

In [3]:
# Order structure to represent each order in the order book
class Order:
    def __init__(self, order_type, symbol, quantity, price, timestamp):
        # "Buy" or "Sell"
        self.order_type = order_type
        self.symbol = symbol
        self.quantity = quantity
        self.price = price
        self.timestamp = timestamp

    def __repr__(self):
        return f"({self.order_type} - {self.symbol} - {self.quantity} @ {self.price})"

In [4]:
# Shared order book with lists for Buy and Sell orders
buy_orders = []
sell_orders = []

In [5]:
# Match orders (Buy and Sell) based on given conditions
def matchOrder():
    """
    The matchOrder function iterates through the Buy and Sell order 
    lists, comparing their prices and symbols to identify matching orders. 
    """
    try:
        i, j = 0, 0
        while i < len(buy_orders) and j < len(sell_orders):
            buy = buy_orders[i]
            sell = sell_orders[j]

            # Match when the buy price >= sell price and both orders are for the same symbol
            if buy.price >= sell.price and buy.symbol == sell.symbol:
                executed_quantity = min(buy.quantity, sell.quantity)
                print(f"Matching Orders: {executed_quantity} shares at {buy.price} for {buy.symbol}")

                # Update quantities or remove orders if fully matched
                if buy.quantity == executed_quantity:
                    # Fully matched buy order
                    i += 1  
                else:
                    buy_orders[i] = Order(buy.order_type, buy.symbol, buy.quantity - executed_quantity, buy.price, buy.timestamp)

                if sell.quantity == executed_quantity:
                    # Fully matched sell order
                    j += 1  
                else:
                    sell_orders[j] = Order(sell.order_type, sell.symbol, sell.quantity - executed_quantity, sell.price, sell.timestamp)

            else:
                # No more matches can occur, so break early
                break  
    except Exception as e:
        log_error(f"Error in function matchOrder: {str(e)}")

In [6]:
# Add an order to the order book
def addOrder(order_type, symbol, quantity, price):
    """
    The addOrder function adds a new order (either "Buy" or "Sell") 
    to the appropriate order list (buy_orders or sell_orders).
    """
    try:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        order = Order(order_type, symbol, quantity, price, timestamp)

        if order_type == "Buy":
            buy_orders.append(order)
            # Sort the buy orders in descending order by price
            buy_orders.sort(key=lambda x: x.price, reverse=True)
        elif order_type == "Sell":
            sell_orders.append(order)
            # Sort the sell orders in ascending order by price
            sell_orders.sort(key=lambda x: x.price)

        print(f"Order Added: {order}")

        # Try to match orders after adding a new one
        matchOrder()

    except Exception as e:
        log_error(f"Error in function addOrder: {str(e)}")

In [7]:
# Simulate random stock transactions (random Buy/Sell orders)
def simulate_activestock_transactions(number_of_transactions, delay_time):
    """
    The simulate_activestock_transactions function generates a specified number of random stock 
    transactions, where each transaction randomly selects whether it's a Buy or Sell order, 
    along with a random ticker symbol, quantity, and price
    """
    try:
        for _ in range(number_of_transactions):
            order_type = "Buy" if random.randint(0, 1) == 0 else "Sell"
            symbol_selected = f"Symbol-{random.randint(1, 1024)}"
            quantity = random.randint(1, 500)
            price = round(random.uniform(10, 300), 2)

            addOrder(order_type, symbol_selected, quantity, price)
            time.sleep(delay_time)
    except Exception as e:
        log_error(f"Error in function simulate_activestock_transactions: {str(e)}")

In [8]:
def simulate_in_threads():
    """
    Simulate the stock transactions in a multi-threaded environment
    """
    try:
        threads = []
        for _ in range(4):  # Create 5 threads simulating different stockbrokers
            t = threading.Thread(target=simulate_activestock_transactions, args=(10, 0.5))
            threads.append(t)
            t.start()

        for t in threads:
            t.join()  # Wait for all threads to finish

    except Exception as e:
        log_error(f"Error in calling function simulate_in_threads: {str(e)}")

In [9]:
# Running the simulation
simulate_in_threads()

Order Added: (Buy - Symbol-621 - 247 @ 11.03)Order Added: (Sell - Symbol-855 - 350 @ 196.41)

Order Added: (Sell - Symbol-299 - 402 @ 52.03)
Order Added: (Sell - Symbol-408 - 452 @ 178.96)
Order Added: (Buy - Symbol-874 - 208 @ 292.6)Order Added: (Sell - Symbol-825 - 466 @ 248.24)

Order Added: (Sell - Symbol-104 - 398 @ 147.68)
Order Added: (Sell - Symbol-807 - 201 @ 55.3)
Order Added: (Buy - Symbol-39 - 16 @ 60.17)Order Added: (Sell - Symbol-509 - 166 @ 213.09)
Order Added: (Sell - Symbol-228 - 401 @ 75.53)

Order Added: (Buy - Symbol-3 - 388 @ 110.45)
Order Added: (Buy - Symbol-886 - 112 @ 52.25)Order Added: (Buy - Symbol-136 - 373 @ 54.6)
Order Added: (Buy - Symbol-690 - 449 @ 58.39)
Order Added: (Buy - Symbol-889 - 114 @ 196.22)

Order Added: (Sell - Symbol-281 - 115 @ 279.15)
Order Added: (Sell - Symbol-809 - 279 @ 233.0)
Order Added: (Sell - Symbol-549 - 500 @ 44.16)
Order Added: (Sell - Symbol-72 - 197 @ 226.76)
Order Added: (Sell - Symbol-436 - 306 @ 248.24)
Order Added: (Buy 

 Also, use lock-free data structures.
 Do not use any dictionaries, maps or equivalent data structures. Essentially there should be no ‘import’-s nor ‘include’-s nor similar construct relevant to the programming language you are using that provides you dictionary, map or equivalent data structure capability. In essence, you are writing the entire code. Standard language-specific non data structure related items are ok, but try to avoid as best as you can.
      Write your ‘matchOrder’ function with a time-complexity of O(n), where 'n' is the number of orders in the Stock order book.